# Section 2: Sparse Matrices, Symmetric Matrices, and Relations/Graphs

In this section we will investigate matrices with special structure and work with our first matrix applications.

In data science and generally matrices can be thought of as a relationship between the columns and the rows. The value in the matrix describes how its column relates to its row. Another math construction for describing relationships is the graph. This makes matrices a powerful representation when working with graph information.

Recall that a graph is a collection of vertices (or nodes) and edges between the vertices. Both the vertices and edges can contain metadata. There are many variations on graphs but for our purpose we will stick to a few rules:
*   Edges must connect two different vertices or create a loop on a single vertex
*   Edges are bidirectional (undirected graph)


## Graph Example

We could create a graph of students at the PSU representing the relationship of having a class together. Students are the vertices. Edges show if two students are in the same class.

We could add even more information to the edges, like the number of classes the students together. Each student would have a self loop edge with the number of classes they are enrolled in.

This example generates a *symmetric* relationship. Justify to yourself why.

## Matrix Representation

Let's see if we can convert this relationship to a matrix. Below I have made a small dataset with students to work with.

If you aren't familiar with python, this is a `dict` (dictionary). A `dict` is a key-value store that allows you to look up values when providing a key. Here the keys are `string`s of students names and the values are `int`s (class IDs). A `dict` cannot have duplicate keys and will overwrite data with multiple assignments.

In [2]:
import numpy as np

# one end goal is going to be making a relationship matrix of students to classes which will show which classes
# any students are taking together

# student paired with class ID of their courses
data = {
    'alice': [0, 1, 6],
    'bob': [2, 7],
    'jack': [2, 4, 5, 1],
    'suzie': [0, 5, 2],
    'chad': [1, 3],
    'karen': [3, 6]
}

We stated that matrices are relations. I can think of a few different relationships from this dataset but let's start with the simplest, student-class. Let's figure out the dimension of this matrix and give the students an enumerated ID.

In [3]:
# empty dictionary
student_ids = dict()

id_counter = 0

#empty set
class_ids_set = set()

for student, class_list in data.items():
    if student not in student_ids:
        student_ids[student] = id_counter
        id_counter += 1
    # using a `dict` with only keys and no values makes it a `set`
    class_ids_set.update(class_list)
    print(class_ids_set)

print(f'student count: {id_counter}')
print('student IDs:')
print(student_ids)
print('class IDs:')
print(class_ids_set)
class_count = len(class_ids_set)
print(f'class count: {class_count}')

"""
we had a dict with student names paired with the list of course id's they are taking
then we made a set of all the possible class ids and a dict of all the students with their id number
"""

{0, 1, 6}
{0, 1, 2, 6, 7}
{0, 1, 2, 4, 5, 6, 7}
{0, 1, 2, 4, 5, 6, 7}
{0, 1, 2, 3, 4, 5, 6, 7}
{0, 1, 2, 3, 4, 5, 6, 7}
student count: 6
student IDs:
{'alice': 0, 'bob': 1, 'jack': 2, 'suzie': 3, 'chad': 4, 'karen': 5}
class IDs:
{0, 1, 2, 3, 4, 5, 6, 7}
class count: 8


"\nwe had a dict with student names paired with the list of course id's they are taking\nthen we made a set of all the possible class ids and a dict of all the students with their id number\n"

We find that our data has 6 students and 8 unique classes.

That means our resulting student to class relationship matrix has dimension of (6, 8).

## Exercise 2.1

Implement the function below that creates this matrix from the student-class data, student IDs, and shape.

Add the data 1.0 into the matrix below where appropriate.

In [4]:
# ideas for storing matrices:
# store each item as a list of triplets [row, col, number]
# that is coo form
# issue is that there are still a lot of repeated things eg. all elements present in row i will store an i

def student_class_matrix(data: dict, student_ids: dict, shape: tuple) -> np.array:
    matrix = np.zeros(shape)
    #for given example, it will be 6x8
    for name in data.keys():
        for class_ID in data[name]:
            matrix[student_ids[name]][class_ID] = 1

    return matrix


A = student_class_matrix(data, student_ids, (id_counter, class_count))
A

array([[1., 1., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 0., 1.],
       [0., 1., 1., 0., 1., 1., 0., 0.],
       [1., 0., 1., 0., 0., 1., 0., 0.],
       [0., 1., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 1., 0.]])

Now that we have the matrix form of the student-class relationship, let's see what happens when we take $AA^T$ or $A^TA$.

In [5]:
A @ A.T

array([[3., 0., 1., 1., 1., 1.],
       [0., 2., 1., 1., 0., 0.],
       [1., 1., 4., 2., 1., 0.],
       [1., 1., 2., 3., 0., 0.],
       [1., 0., 1., 0., 2., 1.],
       [1., 0., 0., 0., 1., 2.]])

In [6]:
A.T @ A

array([[2., 1., 1., 0., 0., 1., 1., 0.],
       [1., 3., 1., 1., 1., 1., 1., 0.],
       [1., 1., 3., 0., 1., 2., 0., 1.],
       [0., 1., 0., 2., 0., 0., 1., 0.],
       [0., 1., 1., 0., 1., 1., 0., 0.],
       [1., 1., 2., 0., 1., 2., 0., 0.],
       [1., 1., 0., 1., 0., 0., 2., 0.],
       [0., 0., 1., 0., 0., 0., 0., 1.]])

## Exercise 2.2

Use this text box to describe in a few sentences the relationship these matrices describe. 

Hint 1: notice the dimension of the resulting matrices.

Hint 2: both of these matrices are *adjacency* matrices as discussed in lecture.

### Response here:

A*A^T is the adjacency matrix of students to students and the number for any pairing is how many courses the two students have together
A^T*A is the adjacency matrix of classes and the number for any pairing is the number of students they have in common

## Sparsity

The three relationship matrices we just generated have a lot of zeros in them. When working with larger sparse datasets, these zeros can be left out to save on memory and algorithm complexity. The three main formats for storing sparse matrices are:
*   `coo` --- coordinate or triplet form
*   `csr` --- compressed sparse row
*   `csc` --- compressed sparse column

but there are also others. Check out the list of [sparse matrix classes](https://docs.scipy.org/doc/scipy/reference/sparse.html) provided by `scipy`.

In lecture, you have talked about the formal description of the three main formats. Note that `scipy`'s specification is slightly different than what you learned in class! At the [bottom of the usage page](https://docs.scipy.org/doc/scipy/reference/sparse.html#further-details) we see:
> CSR column indices are not necessarily sorted. Likewise for CSC row indices. Use the .sorted_indices() and .sort_indices() methods when sorted indices are required

Keep in mind that not all specifications and implementations are the same and reading the specification is the only solution.

## Exercise 2.3

Implement the functions below that convert between dense and sparse formats. Use the matrices from the student-student and student-class relations to check your implementation.

Hint: in the sparse to dense functions below I extract the dimensions of the matrix from the sparse data structure. Understanding why these dimensions make sense is the first step to this exercise.

In [7]:
def dense_to_coo(dense_mat: np.array) -> dict:
    coo = {'rows': [], 'cols': [], 'data': []}
    for i in range(np.shape(dense_mat)[0]):
        for j in range(np.shape(dense_mat)[1]):
            if dense_mat[i][j] != 0:
                coo['rows'].append(i)
                coo['cols'].append(j)
                coo['data'].append(dense_mat[i][j])
    return coo


def dense_to_csr(dense_mat: np.array) -> dict:
    csr = {'cols': [], 'data': [], 'indptr': []}
    csr['indptr'].append(0)
    for i in range(1, np.shape(dense_mat)[0] + 1):
        csr['indptr'].append(0)
        csr['indptr'][i] = csr['indptr'][i - 1]
        for j in range(np.shape(dense_mat)[1]):
            if dense_mat[i - 1][j] != 0:
                csr['cols'].append(j)
                csr['data'].append(dense_mat[i - 1][j])
                csr['indptr'][i] += 1
    return csr


def coo_to_dense(coo_mat: dict) -> np.array:
    num_rows = max(coo_mat['rows']) + 1
    num_cols = max(coo_mat['cols']) + 1
    dense_mat = np.zeros((num_rows, num_cols))
    for i in range(len(coo_mat['rows'])):
        dense_mat[coo_mat['rows'][i]][coo_mat['cols'][i]] = coo_mat['data'][i]
    return dense_mat


def csr_to_dense(csr_mat: dict) -> np.array:
    num_rows = len(csr_mat['indptr']) - 1
    num_cols = max(csr_mat['cols']) + 1
    dense_mat = np.zeros((num_rows, num_cols))
    for i in range(1, num_rows + 1):
        for j in range(csr_mat['indptr'][i - 1], csr_mat['indptr'][i]):
            dense_mat[i - 1][csr_mat['cols'][j]] = csr_mat['data'][j]
    return dense_mat

### Checking Exercise 2.3

Converting from dense to sparse back to dense should have no change.

In [8]:
from numpy.linalg import norm

#### dense -> coo -> dense

In [9]:
coo = dense_to_coo(A)
reconstruction = coo_to_dense(coo)
if norm(A) > 0.0 and norm(A - reconstruction) < 1e-5:
    print('Good job!')
else:
    print('Not quite')
print(A)
#print(coo)
print(reconstruction)


Good job!
[[1. 1. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1.]
 [0. 1. 1. 0. 1. 1. 0. 0.]
 [1. 0. 1. 0. 0. 1. 0. 0.]
 [0. 1. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 1. 0.]]
[[1. 1. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1.]
 [0. 1. 1. 0. 1. 1. 0. 0.]
 [1. 0. 1. 0. 0. 1. 0. 0.]
 [0. 1. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 1. 0.]]



#### dense -> csr -> dense

In [10]:
csr = dense_to_csr(A)
reconstruction = csr_to_dense(csr)
if norm(A) > 0.0 and norm(A - reconstruction) < 1e-5:
    print('Good job!')
else:
    print('Not quite')
print(A)
print(csr)
print(reconstruction)

Good job!
[[1. 1. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1.]
 [0. 1. 1. 0. 1. 1. 0. 0.]
 [1. 0. 1. 0. 0. 1. 0. 0.]
 [0. 1. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 1. 0.]]
{'cols': [0, 1, 6, 2, 7, 1, 2, 4, 5, 0, 2, 5, 1, 3, 3, 6], 'data': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 'indptr': [0, 3, 5, 9, 12, 14, 16]}
[[1. 1. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1.]
 [0. 1. 1. 0. 1. 1. 0. 0.]
 [1. 0. 1. 0. 0. 1. 0. 0.]
 [0. 1. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 1. 0.]]


#### We can also check using `scipy`

Here I convert the dense matrix to the sparse object format provided by `scipy` and access the attributes to create the `dict` version that you built in the previous exercise. Make sure the `dicts` produced by your code matches these.

In [11]:
from scipy import sparse

print('coo:')
coo = sparse.coo_matrix(A)
coo_dict = {'data': coo.data, 'rows': coo.row, 'cols': coo.col}
print(coo_dict)
print(coo)

print('\ncsr:')
csr = sparse.csr_matrix(A)
csr_dict = {'data': csr.data, 'cols': csr.indices, 'indptr': csr.indptr}
print(csr_dict)
print(csr)

coo:
{'data': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]), 'rows': array([0, 0, 0, 1, 1, 2, 2, 2, 2, 3, 3, 3, 4, 4, 5, 5], dtype=int32), 'cols': array([0, 1, 6, 2, 7, 1, 2, 4, 5, 0, 2, 5, 1, 3, 3, 6], dtype=int32)}
  (0, 0)	1.0
  (0, 1)	1.0
  (0, 6)	1.0
  (1, 2)	1.0
  (1, 7)	1.0
  (2, 1)	1.0
  (2, 2)	1.0
  (2, 4)	1.0
  (2, 5)	1.0
  (3, 0)	1.0
  (3, 2)	1.0
  (3, 5)	1.0
  (4, 1)	1.0
  (4, 3)	1.0
  (5, 3)	1.0
  (5, 6)	1.0

csr:
{'data': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]), 'cols': array([0, 1, 6, 2, 7, 1, 2, 4, 5, 0, 2, 5, 1, 3, 3, 6], dtype=int32), 'indptr': array([ 0,  3,  5,  9, 12, 14, 16], dtype=int32)}
  (0, 0)	1.0
  (0, 1)	1.0
  (0, 6)	1.0
  (1, 2)	1.0
  (1, 7)	1.0
  (2, 1)	1.0
  (2, 2)	1.0
  (2, 4)	1.0
  (2, 5)	1.0
  (3, 0)	1.0
  (3, 2)	1.0
  (3, 5)	1.0
  (4, 1)	1.0
  (4, 3)	1.0
  (5, 3)	1.0
  (5, 6)	1.0


## Homework 2

In Homework 1 you were tasked with decomposing a dense system into $A=LU$ and then solving a system $Ax=b$ for $x$ using this decomposition. With sparse matrices, the $LU$ decomposition is rarely used because it likely destroys the sparsity unless you can use knowledge about the structure to select your pivot points.

For this homework, we will load in one of the symmetric matrix martket sparse systems and just throw away the bottom half. The task of solving $Ax=b$ is the same, but never convert $A$ to the dense format. Do all of your work with the `CSR`.

In [12]:
import numpy as np
# from google.colab import drive
from scipy.io import mmread
from scipy import sparse
from scipy.sparse.linalg import spsolve_triangular
from numpy.linalg import norm

# drive.mount('/content/drive')
# NOTE: change this to the folder in your drive with the data
folder = 'mth343/matrices'

In [38]:
coo = mmread(f'{folder}/bcsstk02.mtx')
csr = sparse.triu(coo, format='csr')
b = np.ones(csr.shape[0])

In [41]:
# A is square
# the lower will be completely removed so this function assumes that the input A is an upper triangular matrix
# it will also have to assume a full set of pivots
def solve(A: sparse.csr, b: np.array) -> np.array:
    x = np.zeros(b.shape)
    m = A.shape[0]
    for i in reversed(range(m)):
        #number of column elements in the current row
        num_col_elements = A.indptr[i + 1] - A.indptr[i]

        r = 0
        for col in range(1,num_col_elements):
            r -= A.data[A.indptr[i]+col] * x[A.indices[i]+col]
        x[i] = (b[i] + r)/A.data[A.indptr[i]]
    return x

x = solve(csr, b)
correct_x = spsolve_triangular(csr, b, lower=False, unit_diagonal=False)
'''
print("size x = ", x.shape)
print("size correct x = ", correct_x.shape)
'''

#print("difference = ", x-correct_x)
#print("norm = ", norm(x-correct_x)/norm(correct_x))
if np.allclose(x , correct_x):
    print('Nice work')
else:
    print('Maybe next time')

Nice work
